Este analisis tiene como objetivo identificar patrones y similitudes entre pacientes que presentan el Sindrome de Ovario Poliquistico (SOP), y de igual manera entre los pacientes que no presentan este sindrome. Tambien se pretende generar un modelo de prediccion en base a las variables mas relacionadas con este sindrome, obteniendo una prediccion de clasificacion para cualquier paciente que presente los sintomas que a continuacion analizaremos.
La base de datos utilizada para este analisis fue proporcionado por un Clustering Biomedico el cual se espcializa en areas de la salud y tecnologia. Esta base de datos presentan datos de pacientes de los Estados Unidos, lo cuales muestran algunos datos importantes para la prediccion de SOP.

## Importacion de Librerias

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Importacion de Datos

In [5]:
df = pd.read_csv('PCOS_data.csv')
df.head()

,PCOS (Y/N),Age (yrs),Weight (Kg),Height(Cm),BMI,Blood Group,Pulse rate(bpm),RR (breaths/min),Hb(g/dl),Cycle(R/I),...,Pimples(Y/N),Fast food (Y/N),Reg.Exercise(Y/N),BP _Systolic (mmHg),BP _Diastolic (mmHg),Follicle No. (L),Follicle No. (R),Avg. F size (L) (mm),Avg. F size (R) (mm),Endometrium (mm)
0,0,28,44.6,152.0,19.3,15,78,22,10.48,2,...,0,1.0,0,110,80,3,3,18.0,18.0,8.5
1,0,36,65.0,161.5,24.9,15,74,20,11.70,2,...,0,0.0,0,120,70,3,5,15.0,14.0,3.7
2,1,33,68.8,165.0,25.3,11,72,18,11.80,2,...,1,1.0,0,120,80,13,15,18.0,20.0,10.0
3,0,37,65.0,148.0,29.7,13,72,20,12.00,2,...,0,0.0,0,120,70,2,2,15.0,14.0,7.5
4,0,25,52.0,161.0,20.1,11,72,18,10.00,2,...,0,0.0,0,120,80,3,4,16.0,14.0,7.0


Podemos visualizar las columnas o variables que presenta la base de datos:
Aquí tienes una descripción breve y concisa de cada variable:
### Datos Demográficos y Clínicos

* **PCOS (Y/N):** Variable objetivo. Indica si la paciente tiene diagnóstico de Síndrome de Ovario Poliquístico (Sí/No).
* **Age (yrs):** Edad de la paciente en años.
* **Weight (Kg):** Peso de la paciente en kilogramos.
* **Height(Cm):** Estatura de la paciente en centímetros.
* **BMI:** Índice de Masa Corporal (IMC), calculado a partir del peso y la altura.
* **Blood Group:** Grupo sanguíneo de la paciente (ej. A+, O-).
* **Pulse rate(bpm):** Frecuencia cardíaca (pulsaciones por minuto).
* **RR (breaths/min):** Frecuencia respiratoria (respiraciones por minuto).
* **BP \_Systolic (mmHg):** Presión arterial sistólica (el valor "alto").
* **BP \_Diastolic (mmHg):** Presión arterial diastólica (el valor "bajo").

### Salud Reproductiva e Historial

* **Cycle(R/I):** Indica si el ciclo menstrual es Regular o Irregular.
* **Cycle length(days):** Duración promedio del ciclo menstrual en días.
* **Marraige Status (Yrs):** Duración del matrimonio en años (probablemente usado como indicador de tiempo intentando concebir).
* **Pregnant(Y/N):** Si la paciente está embarazada actualmente (Sí/No).
* **No. of abortions:** Número de abortos previos (espontáneos o inducidos).
* **Follicle No. (L):** Número de folículos contados en el ovario izquierdo (usualmente por ultrasonido).
* **Follicle No. (R):** Número de folículos contados en el ovario derecho.
* **Avg. F size (L) (mm):** Tamaño promedio (en milímetros) de los folículos en el ovario izquierdo.
* **Avg. F size (R) (mm):** Tamaño promedio (en milímetros) de los folículos en el ovario derecho.
* **Endometrium (mm):** Grosor (en milímetros) del endometrio (el revestimiento del útero).

### Análisis de Laboratorio (Hormonas y Sangre)

* **Hb(g/dl):** Nivel de hemoglobina en sangre.
* **I beta-HCG(mIU/mL):** Primer nivel medido de la hormona beta-HCG (hormona del embarazo).
* **II beta-HCG(mIU/mL):** Segundo nivel medido de la beta-HCG (usado para verificar la progresión del embarazo).
* **FSH(mIU/mL):** Nivel de la Hormona Foliculoestimulante (clave en el ciclo ovárico).
* **LH(mIU/mL):** Nivel de la Hormona Luteinizante (clave en la ovulación).
* **FSH/LH:** El ratio o proporción entre los niveles de FSH y LH (relevante para el diagnóstico de SOP).
* **TSH (mIU/L):** Nivel de la Hormona Estimulante de la Tiroides (mide la función tiroidea).
* **AMH(ng/mL):** Nivel de la Hormona Antimülleriana (indicador de la reserva ovárica).
* **PRL(ng/mL):** Nivel de Prolactina (hormona relacionada con la producción de leche).
* **Vit D3 (ng/mL):** Nivel de Vitamina D3 en sangre.
* **PRG(ng/mL):** Nivel de Progesterona (hormona importante en la segunda mitad del ciclo y el embarazo).
* **RBS(mg/dl):** Nivel de glucosa en sangre aleatoria (Random Blood Sugar).

### Síntomas Físicos y Estilo de Vida

* **Hip(inch):** Medida de la circunferencia de la cadera en pulgadas.
* **Waist(inch):** Medida de la circunferencia de la cintura en pulgadas.
* **Waist:Hip Ratio:** Proporción entre la cintura y la cadera (indicador de distribución de grasa).
* **Weight gain(Y/N):** Si la paciente reporta aumento de peso (Sí/No).
* **hair growth(Y/N):** Si la paciente reporta crecimiento excesivo de vello (hirsutismo) (Sí/No).
* **Skin darkening (Y/N):** Si la paciente reporta oscurecimiento de la piel (acantosis nigricans) (Sí/No).
* **Hair loss(Y/N):** Si la paciente reporta caída del cabello (Sí/No).
* **Pimples(Y/N):** Si la paciente reporta tener acné (Sí/No).
* **Fast food (Y/N):** Si la paciente consume comida rápida habitualmente (Sí/No).
* **Reg.Exercise(Y/N):** Si la paciente realiza ejercicio de forma regular (Sí/No).

## Preprocesamiento de Datos

In [8]:
print(df.shape[1])
for i in df.columns:
    print(i)
# df.columns

42
PCOS (Y/N)
 Age (yrs)
Weight (Kg)
Height(Cm) 
BMI
Blood Group
Pulse rate(bpm) 
RR (breaths/min)
Hb(g/dl)
Cycle(R/I)
Cycle length(days)
Marraige Status (Yrs)
Pregnant(Y/N)
No. of abortions
  I   beta-HCG(mIU/mL)
II    beta-HCG(mIU/mL)
FSH(mIU/mL)
LH(mIU/mL)
FSH/LH
Hip(inch)
Waist(inch)
Waist:Hip Ratio
TSH (mIU/L)
AMH(ng/mL)
PRL(ng/mL)
Vit D3 (ng/mL)
PRG(ng/mL)
RBS(mg/dl)
Weight gain(Y/N)
hair growth(Y/N)
Skin darkening (Y/N)
Hair loss(Y/N)
Pimples(Y/N)
Fast food (Y/N)
Reg.Exercise(Y/N)
BP _Systolic (mmHg)
BP _Diastolic (mmHg)
Follicle No. (L)
Follicle No. (R)
Avg. F size (L) (mm)
Avg. F size (R) (mm)
Endometrium (mm)


Podemos ver que algunos nombres de las columnas estan en un formato no adecuado. Es necesario tener un formato adecuado.

In [1]:
df.rename(columns={' Age (yrs)': 'Age (yrs)',
                   'Weight (Kg)': 'Weight (kg)',
                   'Height(Cm)': 'Height (cm)',
                   '  I   beta-HCG(mIU/mL)': 'I beta-HCG (mIU/mL)',
                   'II    beta-HCG(mIU/mL)': 'II beta-HCG(mIU/mL)'},
                   'II    beta-HCG(mIU/mL)': 'II beta-HCG(mIU/mL)'},
          inplace=True)

SyntaxError: closing parenthesis '}' does not match opening parenthesis '(' on line 1 (4035509855.py, line 6)